<a href="https://cognitiveclass.ai/">
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/PY0101EN/Ad/CCLog.png" width="200" align="center">
</a>

<h1 align="center"><font size="5"><b>Interfaz de programación de aplicaciones</b></font></h1>

<p>En este cuaderno aprenderás a convertir un archivo de audio hablado en inglés a texto usando una API de Voz a Texto. Después traducirás la versión en inglés al español usando una API para traducir. <b>Observación:</b> Debes obtener las API Keys para completar el laboratorio.</p>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <a href="https://cocl.us/topNotebooksPython101Coursera">
         <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/PY0101EN/Ad/TopAd.png" width="750" align="center">
    </a>
</div>

<div class="alert alert-block alert-info" style="margin-top: 20px">
<h2>Tabla de Contenido</h2>
<ul>
    <li><a href="#ref0">Convertir Voz a Texto</a></li>
    <li><a href="#ref1">Traductor de Idiomas</a></li>
    <li><a href="#ref2">Ejercicios</a></li>
</ul>
<br>
<p>Tiempo Estimado: <strong>25 min</strong></p>
</div>

In [ ]:
# Necesitarás la siguiente librería
!pip install ibm_watson wget

<h2 id="ref0">Convertir Voz a Texto</h2>

<p>Primero vamos a importar <code>SpeechToTextV1</code> desde <code>ibm_watson</code>. Para mas información sobre la API, por favor visita este <a href="https://cloud.ibm.com/apidocs/speech-to-text?code=python">enlace</a></p>

In [ ]:
from ibm_watson import SpeechToTextV1 
import json
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

<p>El endpoint del servicio esta basado en la ubicación de la instancia, nosotros guardaremos la información en la URL. Para averiguar cual URL usar, revisa los permisos del servicio.</p>

In [ ]:
url_s2t = "https://stream.watsonplatform.net/speech-to-text/api"

<p>Necesitas una API KEY, puedes obtenerla <a href="https://cloud.ibm.com/resources">Aquí </a>.</p>

In [ ]:
iam_apikey_s2t = ""

<p>Creas un <a href="http://watson-developer-cloud.github.io/python-sdk/v0.25.0/apis/watson_developer_cloud.speech_to_text_v1.html">Objeto Speech To Text Adapter</a>. Los parámetros serán el endpoint de la API KEY</p>

In [1]:
authenticator = IAMAuthenticator(iam_apikey_s2t)
s2t = SpeechToTextV1(authenticator=authenticator)
s2t.set_service_url(url_s2t)
s2t

NameError: name 'IAMAuthenticator' is not defined

<p>Vamos a descargar el archivo de audio que usaremos para convertir en texto.</p>

In [ ]:
!wget -O PolynomialRegressionandPipelines.mp3  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/PY0101EN/labs/PolynomialRegressionandPipelines.mp3



<p>Tenemos la ruta al archivo wav que queremos convertir en texto</p>

In [ ]:
filename='PolynomialRegressionandPipelines.mp3'

<p>Creamos el objeto <code>wav</code> con el archivo wav usando <code>open</code> ; configuramos <code>mode</code> a “rb”, esto es parecido al modo lectura y asegura que el archivo esta en modo binario. Usamos el método <code>recognize</code> que regresa el texto reconocido. El parámetro audio es el objeto <code>wav</code>, el parámetro <code>content_type</code> es el formato del archivo de audio.</p>

In [ ]:
with open(filename, mode="rb")  as wav:
    response = s2t.recognize(audio=wav, content_type='audio/mp3')

<p>El atributo “result” contiene un diccionario que incluye la traducción:</p>

In [ ]:
response.result

In [ ]:
from pandas.io.json import json_normalize

json_normalize(response.result['results'],"alternatives")

In [ ]:
response

<p>Obtenemos el texto reconocido y los asignamos a la variable <code>recognized_text</code>:</p>

In [ ]:
recognized_text=response.result['results'][0]["alternatives"][0]["transcript"]
type(recognized_text)

<h2 id="ref1">Traductor de Idiomas</h2>

<p>Primero importamos <code>LanguageTranslatorV3</code> desde ibm_watson. Para mas información sobre la API, por favor visita este <a href="https://cloud.ibm.com/apidocs/speech-to-text?code=python">enlace</a></p>

In [ ]:
from ibm_watson import LanguageTranslatorV3

<p>El endpoint del servicio esta basado en la ubicación de la instancia, nosotros guardaremos la información en la URL. Para averiguar cual URL usar, revisa los permisos del servicio.</p>

In [ ]:
url_lt='https://gateway.watsonplatform.net/language-translator/api'

<p>Necesitas una API KEY, puedes obtenerla <a href="https://cloud.ibm.com/resources">Aquí </a>.</p>

In [ ]:
apikey_lt=''

<p>Las solicitudes a la API requieren un parámetro de la versión que tiene formato = YYYY-MM-DD. Este laboratorio trabaja con la versión actual de Language Translator, 2018-05-01</p>

In [ ]:
version_lt='2018-05-01'

<p>Creamos un objeto tipo Language Translator <code>language_translator</code>:</p>

In [ ]:
authenticator = IAMAuthenticator(apikey_lt)
language_translator = LanguageTranslatorV3(version=version_lt,authenticator=authenticator)
language_translator.set_service_url(url_lt)
language_translator

<p>Podemos obtener una lista del idioma que el servicio puede identificar. El método regresa un código que identifica al idioma. Por ejemplo Inglés (en) a Spanish (es) y el nombre de cada idioma.</p>

In [ ]:
from pandas.io.json import json_normalize

json_normalize(language_translator.list_identifiable_languages().get_result(), "languages")

<p>Podemos usar le método <code>translate</code> para que traduzca el texto. El parámetro text será el texto. Model_id es el tipo de modelo que nos gustaría usar. En este caso, lo configuraremos para ‘en-es’ o Inglés a Español. Obtendremos un objeto tipo Detailed Response <code>translation_response</code>.</p>

In [ ]:
translation_response = language_translator.translate(\
    text=recognized_text, model_id='en-es')
translation_response

<p>El resultado es un diccionario.</p>

In [ ]:
translation=translation_response.get_result()
translation

<p>Podemos obtener la traducción como tipo cadena de la siguiente manera:</p>

In [ ]:
spanish_translation =translation['translations'][0]['translation']
spanish_translation 

<p>Podemos traducir de regreso al idioma inglés</p>

In [ ]:
translation_new = language_translator.translate(text=spanish_translation ,model_id='es-en').get_result()

<p>Podemos obtener la traducción como tipo cadena de la siguiente manera:</p>

In [ ]:
translation_eng=translation_new['translations'][0]['translation']
translation_eng

<p>Podemos traducirlo al idioma frances también:</p>

In [ ]:
French_translation=language_translator.translate(
    text=translation_eng , model_id='en-fr').get_result()

In [ ]:
French_translation['translations'][0]['translation']

<h3>Traductor de Idiomas</h3>

 <a href="http://cocl.us/NotebooksPython101bottom"><img src="https://ibm.box.com/shared/static/irypdxea2q4th88zu1o1tsd06dya10go.png" width="750" align="center"></a>

<b>Referencias</b>

https://cloud.ibm.com/apidocs/speech-to-text?code=python

https://cloud.ibm.com/apidocs/language-translator?code=python

<hr>

<h4>Acerca del Autor:</h4>
<p><a href="https://www.linkedin.com/in/joseph-s-50398b136/">Joseph Santarcangelo</a> posee un doctorado en Ingeniería Eléctrica. Su trabajo de investigación se centra en el uso de Aprendizaje Automático (Machine Learning), Procesamiento de Señales y Visión Artificial para determinar el impacto de los videos en el proceso cognitivo. Joseph trabaja en IBM desde la terminación de su doctorado.</p>

Otros colaboradores: <a href="https://www.linkedin.com/in/fanjiang0619/">Fan Jiang</a>

Copyright &copy; 2019 [cognitiveclass.ai](https:cognitiveclass.ai). This notebook and its source code are released under the terms of the [MIT License](cognitiveclass.ai).